In [ ]:
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
from string import Template
import pandas as pd
import json, random
import yaml
import copy
import networkx as nx
import math
import xml.etree.ElementTree as ET
import ipywidgets as widgets
import os
import time
import os.path
from os import path as p
import glob
from datetime import datetime
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
HTML('''
<script src="../../graph_utils/js/sigma.min.js"></script>
<script src="../../graph_utils/js/sigma.plugins.dragNodes.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.labels.curvedArrow.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.curvedArrow.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.curve.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.labels.curve.js"></script>
<script src="../../graph_utils/js/sigma.canvas.edges.labels.def.js"></script>
<script src="../../graph_utils/js/sigma.exporters.svg.js"></script>
<script src="../../graph_utils/js/settings.js"></script>
<script src="../../graph_utils/js/sigma.renders.snapshots.js"></script>
''')

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display

def on_change(change): # si attiva quando cambio scelta dell'elemento da selezionare
    global attach
    if change['type'] == 'change' and change['name'] == 'value':
        #print ("Stai scegliendo %s" % change['new'])
        attach = change['new']
        return attach

def on_button_update(_):
    clear_output()
    #display(Javascript('''var c = IPython.notebook.get_selected_index();
                       #IPython.notebook.execute_cells([c])'''))
    allegato()

def open_attachment(attach_name):
    path_attachments = "allegati"
    path_file=str(path_attachments + "/" + attach_name)
    ext = os.path.splitext(path_file)[-1].lower()
    #print(path_file)
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        #file = open(path_file, "rb")
        display(Markdown("![alt text](" + path_file + ")"))
        delete = widgets.Button(description='Rimuovi allegato')
        delete.on_click(on_button_delete)
        display(delete)

        #image = file.read()
        #w=widgets.Image(
        #    value=image,
        #    format=ext.replace('.', ''),
        #    width=300,
        #    height=400,
        #)
        #display(w)

def on_button_confirm(_):
    global attach
    clear_output()
    print('Allegato: ' + str(attach))
    open_attachment(attach)

def allegato():
    path_attachments = "allegati"
    global attach # allegato attuale scelto
    update = widgets.Button(description='Aggiorna') #bottone visualizzato in ogni caso
    delete = widgets.Button(description='Rimuovi allegato')
    attach_list = os.listdir(path_attachments)
    if 'ck_points' in attach_list:
        attach_list.remove('ck_points')

    if(attach_list): #si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        confirm = widgets.Button(description='Conferma')
        chosen_attach_wid = widgets.Dropdown(
            options = attach_list,
            value = attach_list[0],
            description = 'Allegati:',
            disabled = False,
        )
        attach = attach_list[0]
        chosen_attach_wid.observe(on_change)
        update.on_click(on_button_update)
        confirm.on_click(on_button_confirm)
        delete.on_click(on_button_delete)
        all_buttons = [update, chosen_attach_wid, confirm, delete]
        display(widgets.HBox(all_buttons))
    else:
        print(f"ATTENZIONE: non sono presenti file nella cartella allegati.") #Ti ricordiamo che la cartella utile per gli allegati da te prodotti per questo esercizio in modo_libero di svolgimento è la cartella @path_ex_folder@/modo_libero/allegati/")
        update.on_click(on_button_update)
        delete.on_click(on_button_delete)
        all_buttons = [update, delete]
        display(widgets.HBox(all_buttons))

def on_button_delete(_):
    #delete_above_cell()
    delete_this_cell()

def delete_above_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c-1).metadata.editable = true;
                                IPython.notebook.get_cell(c-1).metadata.deletable = true;
                                IPython.notebook.delete_cell([c-1]);'''))
def delete_this_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c).metadata.editable = true;
                                IPython.notebook.get_cell(c).metadata.deletable = true;
                                IPython.notebook.delete_cell([c]);'''))

def on_button_raw_attach(_):
    add_attach_cell_din('allegato()')

def on_button_md(_):
    add_md_cell_din()

def on_button_code(_):
    add_code_cell_din()

def on_button_raw(_):
    add_raw_cell_din()

def add_raw_cell_din():
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var idx = IPython.notebook.get_selected_index();
                       var c = IPython.notebook.insert_cell_at_index("raw", idx);
    c.set_text('');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_attach_cell_din(code = ''):
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.editable = false;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_code_cell_din(code = ''):
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id)
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_md_cell_din():
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_cell(code='', position='below', celltype='markdown', is_execute = False):
    """Create a cell in the IPython Notebook.
    code: unicode, Code to fill the new cell with.
    celltype: unicode, Type of cells "code" or "markdown".
    position: unicode, Where to put the cell "below" or "at_bottom"
    is_execute: boolean, To decide if the cell is executed after creation
    """

    # Create a unique id based on epoch time
    display_id = int(time.time()*1e9)

    if is_execute:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        basis.execute();
        """.format(position, celltype, " ")),display_id=display_id)

    else:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        """.format(position, celltype, " ")),display_id=display_id)


    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)

def code_button_delete():
    button_delete = widgets.Button(description="Rimuovi l'allegato", tooltip="Seleziona la cella e clicca su Elimina")
    button_delete.on_click(on_button_delete)
    #print("Vuoi eliminare la cella selezionata?")
    display(button_delete)

def loader_main():
    button_raw_attach = widgets.Button(description="Rispondi (Allegato)", tooltip="Collega un file dalla cartella attachments")
    button_md = widgets.Button(description="Rispondi (Markdown)", abstooltip="Aggiungi una cella per scrivere del testo in Markdown")
    button_code = widgets.Button(description="Rispondi (Code)", tooltip="Aggiungi una cella per scrivere del codice in Python")
    button_raw = widgets.Button(description="Rispondi (Raw)", tooltip="Aggiungi una cella per scrivere del testo libero (SCONSIGLIATA: righe troppo lunghe potrebbero fuoriuscire nella rendition, usala solo se non riesci a controllare il Markdown)")
    button_raw_attach.on_click(on_button_raw_attach)
    button_md.on_click(on_button_md)
    button_code.on_click(on_button_code)
    button_raw.on_click(on_button_raw)

    all_buttons = [button_code, button_md, button_raw, button_raw_attach]
    display(widgets.HBox(all_buttons))


# ESERCIZIO: Flusso massimo in una rete #


Le richieste seguenti fanno tutte riferimento a uno stesso grafo $G = (V,E,w)$ pesato sugli archi. Esso viene riportato più volte negli applet associati alle richieste. Scaricando il grafo in locale, puoi a tua volta sottomettere il numero di copie rieditate che preferisci, nei formati che ti sono più comodi tra quelli proposti.

__Richiesta 1 \[20 punti\]__: Fornire un massimo flusso dal nodo $s$ al nodo $t$

In [ ]:
buttons1 = widgets.Button(description="Avvia task")
buttonl1 = widgets.Button(description="Carica ultima configurazione")
buttonr1 = widgets.Button(description="Reset task")

bllc1 = True
#display(button2, output2)

def run_all1():
    display(Javascript('''
        function sleep2(milliseconds) {
          var start = new Date().getTime();
          for (var i = 0; i < 1e7; i++) {
            if ((new Date().getTime() - start) > milliseconds){
              break;
            }
          }
        }
        var output_area = this;
		// find my cell element
		var cell_element = output_area.element.parents('.cell');
		// which cell is it?
		var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
        try{
            IPython.notebook.execute_cells([cell_idx+1])
			IPython.notebook.execute_cells([cell_idx+2])
			IPython.notebook.execute_cells([cell_idx+3])
        }catch{}
        '''))
    

def on_run_clicked11(b):
    global bllc1
    bllc1=True
    run_all1()
def on_llc1(b):
    global bllc1
    bllc1=False
    run_all1()

def on_reset_clicked1(b):
    global bllc1
    bllc1=True
    run_all1()    
    
buttons1.on_click(on_run_clicked11)
buttonl1.on_click(on_llc1)
buttonr1.on_click(on_reset_clicked1)
widgets.HBox([buttons1, buttonl1, buttonr1])

In [ ]:

    graph_data1 = {'nodes': [{'id': 's', 'label': 's', 'x': 776.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'g', 'label': 'g', 'x': 970.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'e', 'label': 'e', 'x': 826.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'f', 'label': 'f', 'x': 930.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'c', 'label': 'c', 'x': 928.75, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'd', 'label': 'd', 'x': 824.25, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'a', 'label': 'a', 'x': 774.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'b', 'label': 'b', 'x': 968.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'o', 'label': 'o', 'x': 1087.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'n', 'label': 'n', 'x': 1294.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'm', 'label': 'm', 'x': 1142.625, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'l', 'label': 'l', 'x': 1240.125, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'i', 'label': 'i', 'x': 1140.625, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'h', 'label': 'h', 'x': 1238.125, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'q', 'label': 'q', 'x': 1085.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'p', 'label': 'p', 'x': 1292.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'w', 'label': 'w', 'x': 1461.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 't', 'label': 't', 'x': 1655.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'v', 'label': 'v', 'x': 1511.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'z', 'label': 'z', 'x': 1615.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'r', 'label': 'r', 'x': 1509.25, 'y': -66.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'u', 'label': 'u', 'x': 1613.75, 'y': -68.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'x', 'label': 'x', 'x': 1459.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'y', 'label': 'y', 'x': 1653.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'hidden0', 'label': '', 'x': 1245.25, 'y': 90.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden1', 'label': '', 'x': 1421.5, 'y': 18.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden2', 'label': '', 'x': 1047.25, 'y': 105.75, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}], 'edges': [{'id': 'e0', 'source': 's', 'target': 'g', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e1', 'source': 'e', 'target': 'f', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e2', 'source': 'f', 'target': 'd', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e3', 'source': 'd', 'target': 'c', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e4', 'source': 'c', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e5', 'source': 's', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e6', 'source': 'g', 'target': 'f', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e7', 'source': 'g', 'target': 'b', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e8', 'source': 's', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e9', 'source': 'a', 'target': 'b', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e10', 'source': 'd', 'target': 'b', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e11', 'source': 'c', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e12', 'source': 'o', 'target': 'm', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e13', 'source': 'i', 'target': 'l', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e14', 'source': 'q', 'target': 'i', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e15', 'source': 'n', 'target': 'l', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e16', 'source': 'm', 'target': 'h', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e17', 'source': 'p', 'target': 'h', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e18', 'source': 'n', 'target': 'p', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e19', 'source': 'o', 'target': 'q', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e20', 'source': 'b', 'target': 'q', 'type': 'line', 'label': '12', 'weight': '12', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e21', 'source': 'g', 'target': 'o', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e22', 'source': 'p', 'target': 'x', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e23', 'source': 'n', 'target': 'w', 'type': 'line', 'label': '14', 'weight': '14', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e24', 'source': 'w', 'target': 'x', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e25', 'source': 'v', 'target': 'r', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e26', 'source': 'r', 'target': 'u', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e27', 'source': 'u', 'target': 'z', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e28', 'source': 'z', 'target': 'v', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e29', 'source': 'w', 'target': 'v', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e30', 'source': 'u', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e31', 'source': 'x', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e32', 'source': 'y', 'target': 't', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e33', 'source': 'z', 'target': 't', 'type': 'line', 'label': '9', 'weight': '9', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e34', 'source': 'x', 'target': 'r', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e35', 'source': 'w', 'target': 't', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e36', 'source': 'm', 'target': 'i', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e37', 'source': 'l', 'target': 'h', 'type': None, 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'h0', 'source': 'q', 'target': 'hidden0', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h1', 'source': 'hidden0', 'target': 'hidden1', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h2', 'source': 'hidden1', 'target': 'n', 'type': 'arrow', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h3', 'source': 'o', 'target': 'hidden2', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h4', 'source': 'hidden2', 'target': 'p', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}]}
    friends_e1 = {'h0': ['h0', 'h1', 'h2'], 'h1': ['h0', 'h1', 'h2'], 'h2': ['h0', 'h1', 'h2'], 'h3': ['h3', 'h4'], 'h4': ['h3', 'h4']}
    friends_n1 = {'q': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden0': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden1': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'n': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'o': ['o', 'hidden2', 'hidden2', 'p'], 'hidden2': ['o', 'hidden2', 'hidden2', 'p', 'o', 'hidden2', 'hidden2', 'p'], 'p': ['o', 'hidden2', 'hidden2', 'p']}
    center1 = (388.375, -125.25)
    html_text1 = '''<div>
    <button class="button" id="btn_set_one1">Colora</button>
</div>
<div>
    <button class="button" id="btn_set_two1">Colora</button>
</div>
<div>
    <button class="button" id="btn_unselect_set1">Scolora</button>
</div>
<div>
    <button class="button" id="btn_multiple_sel1">Selezione multipla</button>
</div>
<div>
    <button class="button" id="download_png1">Download Png</button>
</div>
<div>
    <button class="button" id="download_svg1">Download Svg</button>
</div>
<div>
    <button class="button" id="download_adj1">Download Matrice di Adiacenza</button>
</div>
<div>
    <button class="button" id="save1">Salva</button>
</div>'''
    

In [ ]:
#parsing del grafo xml o graphml, viene creato graph_data, original_nodes e original_edges
if bllc1:
    pass
else:
    try:
        list_of_files1 = glob.glob('allegati/ck_points/graph_data_'+str(1)+'_*')
        latest_file1 = max(list_of_files1, key=os.path.getctime)
    except:
        pass
    try:
        with open(latest_file1) as json_file:     
            graph_data1 = json.load(json_file)
    except:
        print("Nessun file di configurazione trovato!")

In [ ]:
js_text_template1 = Template(open('../../graph_utils/js/sigma-graph.js','r').read())
#faccio il dump della struttura che ho creato dentro il js
js_text1 = js_text_template1.substitute({'graph_data': json.dumps(graph_data1),
                                       'exercise' : 1,
                                      'container':'sigma-container1',
                                      'data':0,
                                       'x':center1[0]+center1[0]*0.6,
                                       'y':center1[1],
                                      's':'s1',
                                     'n':'n1',
                                      'ef':'`'+str(friends_e1)+'`',
                                      'nf':'`'+str(friends_n1)+'`',
                                       's_cell': 'cell1'})

html_template1 = Template('''
<style>
    .container { width:100% !important; }
</style>
<link rel="stylesheet" href="../../graph_utils/css/style.css">

    <div class="row row-no-gutters" style="border-color: #1e6add; border-width: 2px;">
        <div class="col-sm-1" style="border-color: #1e6add; border-width: 2px;">
            ''' + html_text1 + '''
        </div>
        <div class="col-sm-11" style="position:border-color: #1e6add; border-width: 2px;">
            <div id="sigma-container1" style="height:750px; width:100%"></div>
        </div>
    </div>

<script> $js_text1 </script>
''')
#render HTML con sostituzione variabili
HTML(html_template1.substitute({'js_text1': js_text1}))

In [ ]:
loader_main()


__Richiesta 2 \[15 punti\]__: Certificare l'ottimalità del flusso massimo dal nodo $s$ al nodo $t$

In [ ]:
buttons2 = widgets.Button(description="Avvia task")
buttonl2 = widgets.Button(description="Carica ultima configurazione")
buttonr2 = widgets.Button(description="Reset task")

bllc2 = True
#display(button2, output2)

def run_all2():
    display(Javascript('''
        function sleep2(milliseconds) {
          var start = new Date().getTime();
          for (var i = 0; i < 1e7; i++) {
            if ((new Date().getTime() - start) > milliseconds){
              break;
            }
          }
        }
        var output_area = this;
		// find my cell element
		var cell_element = output_area.element.parents('.cell');
		// which cell is it?
		var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
        try{
            IPython.notebook.execute_cells([cell_idx+1])
			IPython.notebook.execute_cells([cell_idx+2])
			IPython.notebook.execute_cells([cell_idx+3])
        }catch{}
        '''))
    

def on_run_clicked12(b):
    global bllc2
    bllc2=True
    run_all2()
def on_llc2(b):
    global bllc2
    bllc2=False
    run_all2()

def on_reset_clicked2(b):
    global bllc2
    bllc2=True
    run_all2()    
    
buttons2.on_click(on_run_clicked12)
buttonl2.on_click(on_llc2)
buttonr2.on_click(on_reset_clicked2)
widgets.HBox([buttons2, buttonl2, buttonr2])

In [ ]:

    graph_data2 = {'nodes': [{'id': 's', 'label': 's', 'x': 776.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'g', 'label': 'g', 'x': 970.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'e', 'label': 'e', 'x': 826.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'f', 'label': 'f', 'x': 930.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'c', 'label': 'c', 'x': 928.75, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'd', 'label': 'd', 'x': 824.25, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'a', 'label': 'a', 'x': 774.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'b', 'label': 'b', 'x': 968.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'o', 'label': 'o', 'x': 1087.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'n', 'label': 'n', 'x': 1294.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'm', 'label': 'm', 'x': 1142.625, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'l', 'label': 'l', 'x': 1240.125, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'i', 'label': 'i', 'x': 1140.625, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'h', 'label': 'h', 'x': 1238.125, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'q', 'label': 'q', 'x': 1085.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'p', 'label': 'p', 'x': 1292.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'w', 'label': 'w', 'x': 1461.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 't', 'label': 't', 'x': 1655.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'v', 'label': 'v', 'x': 1511.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'z', 'label': 'z', 'x': 1615.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'r', 'label': 'r', 'x': 1509.25, 'y': -66.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'u', 'label': 'u', 'x': 1613.75, 'y': -68.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'x', 'label': 'x', 'x': 1459.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'y', 'label': 'y', 'x': 1653.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'hidden0', 'label': '', 'x': 1245.25, 'y': 90.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden1', 'label': '', 'x': 1421.5, 'y': 18.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden2', 'label': '', 'x': 1047.25, 'y': 105.75, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}], 'edges': [{'id': 'e0', 'source': 's', 'target': 'g', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e1', 'source': 'e', 'target': 'f', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e2', 'source': 'f', 'target': 'd', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e3', 'source': 'd', 'target': 'c', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e4', 'source': 'c', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e5', 'source': 's', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e6', 'source': 'g', 'target': 'f', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e7', 'source': 'g', 'target': 'b', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e8', 'source': 's', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e9', 'source': 'a', 'target': 'b', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e10', 'source': 'd', 'target': 'b', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e11', 'source': 'c', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e12', 'source': 'o', 'target': 'm', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e13', 'source': 'i', 'target': 'l', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e14', 'source': 'q', 'target': 'i', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e15', 'source': 'n', 'target': 'l', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e16', 'source': 'm', 'target': 'h', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e17', 'source': 'p', 'target': 'h', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e18', 'source': 'n', 'target': 'p', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e19', 'source': 'o', 'target': 'q', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e20', 'source': 'b', 'target': 'q', 'type': 'line', 'label': '12', 'weight': '12', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e21', 'source': 'g', 'target': 'o', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e22', 'source': 'p', 'target': 'x', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e23', 'source': 'n', 'target': 'w', 'type': 'line', 'label': '14', 'weight': '14', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e24', 'source': 'w', 'target': 'x', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e25', 'source': 'v', 'target': 'r', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e26', 'source': 'r', 'target': 'u', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e27', 'source': 'u', 'target': 'z', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e28', 'source': 'z', 'target': 'v', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e29', 'source': 'w', 'target': 'v', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e30', 'source': 'u', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e31', 'source': 'x', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e32', 'source': 'y', 'target': 't', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e33', 'source': 'z', 'target': 't', 'type': 'line', 'label': '9', 'weight': '9', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e34', 'source': 'x', 'target': 'r', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e35', 'source': 'w', 'target': 't', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e36', 'source': 'm', 'target': 'i', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e37', 'source': 'l', 'target': 'h', 'type': None, 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'h0', 'source': 'q', 'target': 'hidden0', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h1', 'source': 'hidden0', 'target': 'hidden1', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h2', 'source': 'hidden1', 'target': 'n', 'type': 'arrow', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h3', 'source': 'o', 'target': 'hidden2', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h4', 'source': 'hidden2', 'target': 'p', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}]}
    friends_e2 = {'h0': ['h0', 'h1', 'h2'], 'h1': ['h0', 'h1', 'h2'], 'h2': ['h0', 'h1', 'h2'], 'h3': ['h3', 'h4'], 'h4': ['h3', 'h4']}
    friends_n2 = {'q': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden0': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden1': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'n': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'o': ['o', 'hidden2', 'hidden2', 'p'], 'hidden2': ['o', 'hidden2', 'hidden2', 'p', 'o', 'hidden2', 'hidden2', 'p'], 'p': ['o', 'hidden2', 'hidden2', 'p']}
    center2 = (388.375, -125.25)
    html_text2 = '''<div>
    <button class="button" id="btn_set_one2">Colora</button>
</div>
<div>
    <button class="button" id="btn_set_two2">Colora</button>
</div>
<div>
    <button class="button" id="btn_unselect_set2">Scolora</button>
</div>
<div>
    <button class="button" id="btn_multiple_sel2">Selezione multipla</button>
</div>
<div>
    <button class="button" id="download_png2">Download Png</button>
</div>
<div>
    <button class="button" id="download_svg2">Download Svg</button>
</div>
<div>
    <button class="button" id="download_adj2">Download Matrice di Adiacenza</button>
</div>
<div>
    <button class="button" id="save2">Salva</button>
</div>'''
    

In [ ]:
#parsing del grafo xml o graphml, viene creato graph_data, original_nodes e original_edges
if bllc2:
    pass
else:
    try:
        list_of_files2 = glob.glob('allegati/ck_points/graph_data_'+str(2)+'_*')
        latest_file2 = max(list_of_files2, key=os.path.getctime)
    except:
        pass
    try:
        with open(latest_file2) as json_file:     
            graph_data2 = json.load(json_file)
    except:
        print("Nessun file di configurazione trovato!")

In [ ]:
js_text_template2 = Template(open('../../graph_utils/js/sigma-graph.js','r').read())
#faccio il dump della struttura che ho creato dentro il js
js_text2 = js_text_template2.substitute({'graph_data': json.dumps(graph_data2),
                                       'exercise' : 2,
                                      'container':'sigma-container2',
                                      'data':0,
                                       'x':center2[0]+center2[0]*0.6,
                                       'y':center2[1],
                                      's':'s2',
                                     'n':'n2',
                                      'ef':'`'+str(friends_e2)+'`',
                                      'nf':'`'+str(friends_n2)+'`',
                                       's_cell': 'cell2'})

html_template2 = Template('''
<style>
    .container { width:100% !important; }
</style>
<link rel="stylesheet" href="../../graph_utils/css/style.css">

    <div class="row row-no-gutters" style="border-color: #1e6add; border-width: 2px;">
        <div class="col-sm-1" style="border-color: #1e6add; border-width: 2px;">
            ''' + html_text2 + '''
        </div>
        <div class="col-sm-11" style="position:border-color: #1e6add; border-width: 2px;">
            <div id="sigma-container2" style="height:750px; width:100%"></div>
        </div>
    </div>

<script> $js_text2 </script>
''')
#render HTML con sostituzione variabili
HTML(html_template2.substitute({'js_text2': js_text2}))

In [ ]:
loader_main()


__Richiesta 3 \[5 punti\]__: Dire, certificandolo, se l'arco $hl$ appartenga a tutti, oppure a nessuno, oppure a qualcuno ma non tutti gli $s,t$-tagli di capacità minima

In [ ]:
buttons3 = widgets.Button(description="Avvia task")
buttonl3 = widgets.Button(description="Carica ultima configurazione")
buttonr3 = widgets.Button(description="Reset task")

bllc3 = True
#display(button2, output2)

def run_all3():
    display(Javascript('''
        function sleep2(milliseconds) {
          var start = new Date().getTime();
          for (var i = 0; i < 1e7; i++) {
            if ((new Date().getTime() - start) > milliseconds){
              break;
            }
          }
        }
        var output_area = this;
		// find my cell element
		var cell_element = output_area.element.parents('.cell');
		// which cell is it?
		var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
        try{
            IPython.notebook.execute_cells([cell_idx+1])
			IPython.notebook.execute_cells([cell_idx+2])
			IPython.notebook.execute_cells([cell_idx+3])
        }catch{}
        '''))
    

def on_run_clicked13(b):
    global bllc3
    bllc3=True
    run_all3()
def on_llc3(b):
    global bllc3
    bllc3=False
    run_all3()

def on_reset_clicked3(b):
    global bllc3
    bllc3=True
    run_all3()    
    
buttons3.on_click(on_run_clicked13)
buttonl3.on_click(on_llc3)
buttonr3.on_click(on_reset_clicked3)
widgets.HBox([buttons3, buttonl3, buttonr3])

In [ ]:

    graph_data3 = {'nodes': [{'id': 's', 'label': 's', 'x': 776.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'g', 'label': 'g', 'x': 970.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'e', 'label': 'e', 'x': 826.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'f', 'label': 'f', 'x': 930.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'c', 'label': 'c', 'x': 928.75, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'd', 'label': 'd', 'x': 824.25, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'a', 'label': 'a', 'x': 774.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'b', 'label': 'b', 'x': 968.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'o', 'label': 'o', 'x': 1087.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'n', 'label': 'n', 'x': 1294.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'm', 'label': 'm', 'x': 1142.625, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'l', 'label': 'l', 'x': 1240.125, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'i', 'label': 'i', 'x': 1140.625, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'h', 'label': 'h', 'x': 1238.125, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'q', 'label': 'q', 'x': 1085.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'p', 'label': 'p', 'x': 1292.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'w', 'label': 'w', 'x': 1461.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 't', 'label': 't', 'x': 1655.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'v', 'label': 'v', 'x': 1511.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'z', 'label': 'z', 'x': 1615.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'r', 'label': 'r', 'x': 1509.25, 'y': -66.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'u', 'label': 'u', 'x': 1613.75, 'y': -68.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'x', 'label': 'x', 'x': 1459.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'y', 'label': 'y', 'x': 1653.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'hidden0', 'label': '', 'x': 1245.25, 'y': 90.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden1', 'label': '', 'x': 1421.5, 'y': 18.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden2', 'label': '', 'x': 1047.25, 'y': 105.75, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}], 'edges': [{'id': 'e0', 'source': 's', 'target': 'g', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e1', 'source': 'e', 'target': 'f', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e2', 'source': 'f', 'target': 'd', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e3', 'source': 'd', 'target': 'c', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e4', 'source': 'c', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e5', 'source': 's', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e6', 'source': 'g', 'target': 'f', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e7', 'source': 'g', 'target': 'b', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e8', 'source': 's', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e9', 'source': 'a', 'target': 'b', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e10', 'source': 'd', 'target': 'b', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e11', 'source': 'c', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e12', 'source': 'o', 'target': 'm', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e13', 'source': 'i', 'target': 'l', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e14', 'source': 'q', 'target': 'i', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e15', 'source': 'n', 'target': 'l', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e16', 'source': 'm', 'target': 'h', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e17', 'source': 'p', 'target': 'h', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e18', 'source': 'n', 'target': 'p', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e19', 'source': 'o', 'target': 'q', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e20', 'source': 'b', 'target': 'q', 'type': 'line', 'label': '12', 'weight': '12', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e21', 'source': 'g', 'target': 'o', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e22', 'source': 'p', 'target': 'x', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e23', 'source': 'n', 'target': 'w', 'type': 'line', 'label': '14', 'weight': '14', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e24', 'source': 'w', 'target': 'x', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e25', 'source': 'v', 'target': 'r', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e26', 'source': 'r', 'target': 'u', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e27', 'source': 'u', 'target': 'z', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e28', 'source': 'z', 'target': 'v', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e29', 'source': 'w', 'target': 'v', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e30', 'source': 'u', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e31', 'source': 'x', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e32', 'source': 'y', 'target': 't', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e33', 'source': 'z', 'target': 't', 'type': 'line', 'label': '9', 'weight': '9', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e34', 'source': 'x', 'target': 'r', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e35', 'source': 'w', 'target': 't', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e36', 'source': 'm', 'target': 'i', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e37', 'source': 'l', 'target': 'h', 'type': None, 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'h0', 'source': 'q', 'target': 'hidden0', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h1', 'source': 'hidden0', 'target': 'hidden1', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h2', 'source': 'hidden1', 'target': 'n', 'type': 'arrow', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h3', 'source': 'o', 'target': 'hidden2', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h4', 'source': 'hidden2', 'target': 'p', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}]}
    friends_e3 = {'h0': ['h0', 'h1', 'h2'], 'h1': ['h0', 'h1', 'h2'], 'h2': ['h0', 'h1', 'h2'], 'h3': ['h3', 'h4'], 'h4': ['h3', 'h4']}
    friends_n3 = {'q': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden0': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden1': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'n': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'o': ['o', 'hidden2', 'hidden2', 'p'], 'hidden2': ['o', 'hidden2', 'hidden2', 'p', 'o', 'hidden2', 'hidden2', 'p'], 'p': ['o', 'hidden2', 'hidden2', 'p']}
    center3 = (388.375, -125.25)
    html_text3 = '''<div>
    <button class="button" id="btn_set_one3">Colora</button>
</div>
<div>
    <button class="button" id="btn_set_two3">Colora</button>
</div>
<div>
    <button class="button" id="btn_unselect_set3">Scolora</button>
</div>
<div>
    <button class="button" id="btn_multiple_sel3">Selezione multipla</button>
</div>
<div>
    <button class="button" id="download_png3">Download Png</button>
</div>
<div>
    <button class="button" id="download_svg3">Download Svg</button>
</div>
<div>
    <button class="button" id="download_adj3">Download Matrice di Adiacenza</button>
</div>
<div>
    <button class="button" id="save3">Salva</button>
</div>'''
    

In [ ]:
#parsing del grafo xml o graphml, viene creato graph_data, original_nodes e original_edges
if bllc3:
    pass
else:
    try:
        list_of_files3 = glob.glob('allegati/ck_points/graph_data_'+str(3)+'_*')
        latest_file3 = max(list_of_files3, key=os.path.getctime)
    except:
        pass
    try:
        with open(latest_file3) as json_file:     
            graph_data3 = json.load(json_file)
    except:
        print("Nessun file di configurazione trovato!")

In [ ]:
js_text_template3 = Template(open('../../graph_utils/js/sigma-graph.js','r').read())
#faccio il dump della struttura che ho creato dentro il js
js_text3 = js_text_template3.substitute({'graph_data': json.dumps(graph_data3),
                                       'exercise' : 3,
                                      'container':'sigma-container3',
                                      'data':0,
                                       'x':center3[0]+center3[0]*0.6,
                                       'y':center3[1],
                                      's':'s3',
                                     'n':'n3',
                                      'ef':'`'+str(friends_e3)+'`',
                                      'nf':'`'+str(friends_n3)+'`',
                                       's_cell': 'cell3'})

html_template3 = Template('''
<style>
    .container { width:100% !important; }
</style>
<link rel="stylesheet" href="../../graph_utils/css/style.css">

    <div class="row row-no-gutters" style="border-color: #1e6add; border-width: 2px;">
        <div class="col-sm-1" style="border-color: #1e6add; border-width: 2px;">
            ''' + html_text3 + '''
        </div>
        <div class="col-sm-11" style="position:border-color: #1e6add; border-width: 2px;">
            <div id="sigma-container3" style="height:750px; width:100%"></div>
        </div>
    </div>

<script> $js_text3 </script>
''')
#render HTML con sostituzione variabili
HTML(html_template3.substitute({'js_text3': js_text3}))

In [ ]:
loader_main()


__Richiesta 4 \[5 punti\]__: Dire, certificandolo, se l'arco $mo$ appartenga a tutti, oppure a nessuno, oppure a qualcuno ma non tutti gli $s,t$-tagli di capacità minima

In [ ]:
buttons4 = widgets.Button(description="Avvia task")
buttonl4 = widgets.Button(description="Carica ultima configurazione")
buttonr4 = widgets.Button(description="Reset task")

bllc4 = True
#display(button2, output2)

def run_all4():
    display(Javascript('''
        function sleep2(milliseconds) {
          var start = new Date().getTime();
          for (var i = 0; i < 1e7; i++) {
            if ((new Date().getTime() - start) > milliseconds){
              break;
            }
          }
        }
        var output_area = this;
		// find my cell element
		var cell_element = output_area.element.parents('.cell');
		// which cell is it?
		var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
        try{
            IPython.notebook.execute_cells([cell_idx+1])
			IPython.notebook.execute_cells([cell_idx+2])
			IPython.notebook.execute_cells([cell_idx+3])
        }catch{}
        '''))
    

def on_run_clicked14(b):
    global bllc4
    bllc4=True
    run_all4()
def on_llc4(b):
    global bllc4
    bllc4=False
    run_all4()

def on_reset_clicked4(b):
    global bllc4
    bllc4=True
    run_all4()    
    
buttons4.on_click(on_run_clicked14)
buttonl4.on_click(on_llc4)
buttonr4.on_click(on_reset_clicked4)
widgets.HBox([buttons4, buttonl4, buttonr4])

In [ ]:

    graph_data4 = {'nodes': [{'id': 's', 'label': 's', 'x': 776.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'g', 'label': 'g', 'x': 970.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'e', 'label': 'e', 'x': 826.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'f', 'label': 'f', 'x': 930.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'c', 'label': 'c', 'x': 928.75, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'd', 'label': 'd', 'x': 824.25, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'a', 'label': 'a', 'x': 774.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'b', 'label': 'b', 'x': 968.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'o', 'label': 'o', 'x': 1087.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'n', 'label': 'n', 'x': 1294.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'm', 'label': 'm', 'x': 1142.625, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'l', 'label': 'l', 'x': 1240.125, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'i', 'label': 'i', 'x': 1140.625, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'h', 'label': 'h', 'x': 1238.125, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'q', 'label': 'q', 'x': 1085.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'p', 'label': 'p', 'x': 1292.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'w', 'label': 'w', 'x': 1461.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 't', 'label': 't', 'x': 1655.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'v', 'label': 'v', 'x': 1511.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'z', 'label': 'z', 'x': 1615.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'r', 'label': 'r', 'x': 1509.25, 'y': -66.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'u', 'label': 'u', 'x': 1613.75, 'y': -68.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'x', 'label': 'x', 'x': 1459.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'y', 'label': 'y', 'x': 1653.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'hidden0', 'label': '', 'x': 1245.25, 'y': 90.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden1', 'label': '', 'x': 1421.5, 'y': 18.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden2', 'label': '', 'x': 1047.25, 'y': 105.75, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}], 'edges': [{'id': 'e0', 'source': 's', 'target': 'g', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e1', 'source': 'e', 'target': 'f', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e2', 'source': 'f', 'target': 'd', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e3', 'source': 'd', 'target': 'c', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e4', 'source': 'c', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e5', 'source': 's', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e6', 'source': 'g', 'target': 'f', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e7', 'source': 'g', 'target': 'b', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e8', 'source': 's', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e9', 'source': 'a', 'target': 'b', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e10', 'source': 'd', 'target': 'b', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e11', 'source': 'c', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e12', 'source': 'o', 'target': 'm', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e13', 'source': 'i', 'target': 'l', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e14', 'source': 'q', 'target': 'i', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e15', 'source': 'n', 'target': 'l', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e16', 'source': 'm', 'target': 'h', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e17', 'source': 'p', 'target': 'h', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e18', 'source': 'n', 'target': 'p', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e19', 'source': 'o', 'target': 'q', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e20', 'source': 'b', 'target': 'q', 'type': 'line', 'label': '12', 'weight': '12', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e21', 'source': 'g', 'target': 'o', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e22', 'source': 'p', 'target': 'x', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e23', 'source': 'n', 'target': 'w', 'type': 'line', 'label': '14', 'weight': '14', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e24', 'source': 'w', 'target': 'x', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e25', 'source': 'v', 'target': 'r', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e26', 'source': 'r', 'target': 'u', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e27', 'source': 'u', 'target': 'z', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e28', 'source': 'z', 'target': 'v', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e29', 'source': 'w', 'target': 'v', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e30', 'source': 'u', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e31', 'source': 'x', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e32', 'source': 'y', 'target': 't', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e33', 'source': 'z', 'target': 't', 'type': 'line', 'label': '9', 'weight': '9', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e34', 'source': 'x', 'target': 'r', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e35', 'source': 'w', 'target': 't', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e36', 'source': 'm', 'target': 'i', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e37', 'source': 'l', 'target': 'h', 'type': None, 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'h0', 'source': 'q', 'target': 'hidden0', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h1', 'source': 'hidden0', 'target': 'hidden1', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h2', 'source': 'hidden1', 'target': 'n', 'type': 'arrow', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h3', 'source': 'o', 'target': 'hidden2', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h4', 'source': 'hidden2', 'target': 'p', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}]}
    friends_e4 = {'h0': ['h0', 'h1', 'h2'], 'h1': ['h0', 'h1', 'h2'], 'h2': ['h0', 'h1', 'h2'], 'h3': ['h3', 'h4'], 'h4': ['h3', 'h4']}
    friends_n4 = {'q': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden0': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden1': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'n': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'o': ['o', 'hidden2', 'hidden2', 'p'], 'hidden2': ['o', 'hidden2', 'hidden2', 'p', 'o', 'hidden2', 'hidden2', 'p'], 'p': ['o', 'hidden2', 'hidden2', 'p']}
    center4 = (388.375, -125.25)
    html_text4 = '''<div>
    <button class="button" id="btn_set_one4">Colora</button>
</div>
<div>
    <button class="button" id="btn_set_two4">Colora</button>
</div>
<div>
    <button class="button" id="btn_unselect_set4">Scolora</button>
</div>
<div>
    <button class="button" id="btn_multiple_sel4">Selezione multipla</button>
</div>
<div>
    <button class="button" id="download_png4">Download Png</button>
</div>
<div>
    <button class="button" id="download_svg4">Download Svg</button>
</div>
<div>
    <button class="button" id="download_adj4">Download Matrice di Adiacenza</button>
</div>
<div>
    <button class="button" id="save4">Salva</button>
</div>'''
    

In [ ]:
#parsing del grafo xml o graphml, viene creato graph_data, original_nodes e original_edges
if bllc4:
    pass
else:
    try:
        list_of_files4 = glob.glob('allegati/ck_points/graph_data_'+str(4)+'_*')
        latest_file4 = max(list_of_files4, key=os.path.getctime)
    except:
        pass
    try:
        with open(latest_file4) as json_file:     
            graph_data4 = json.load(json_file)
    except:
        print("Nessun file di configurazione trovato!")

In [ ]:
js_text_template4 = Template(open('../../graph_utils/js/sigma-graph.js','r').read())
#faccio il dump della struttura che ho creato dentro il js
js_text4 = js_text_template4.substitute({'graph_data': json.dumps(graph_data4),
                                       'exercise' : 4,
                                      'container':'sigma-container4',
                                      'data':0,
                                       'x':center4[0]+center4[0]*0.6,
                                       'y':center4[1],
                                      's':'s4',
                                     'n':'n4',
                                      'ef':'`'+str(friends_e4)+'`',
                                      'nf':'`'+str(friends_n4)+'`',
                                       's_cell': 'cell4'})

html_template4 = Template('''
<style>
    .container { width:100% !important; }
</style>
<link rel="stylesheet" href="../../graph_utils/css/style.css">

    <div class="row row-no-gutters" style="border-color: #1e6add; border-width: 2px;">
        <div class="col-sm-1" style="border-color: #1e6add; border-width: 2px;">
            ''' + html_text4 + '''
        </div>
        <div class="col-sm-11" style="position:border-color: #1e6add; border-width: 2px;">
            <div id="sigma-container4" style="height:750px; width:100%"></div>
        </div>
    </div>

<script> $js_text4 </script>
''')
#render HTML con sostituzione variabili
HTML(html_template4.substitute({'js_text4': js_text4}))

In [ ]:
loader_main()


__Richiesta 5 \[5 punti\]__: Dire, certificandolo, se l'arco $wx$ appartenga a tutti, oppure a nessuno, oppure a qualcuno ma non tutti gli $s,t$-tagli di capacità minima

In [ ]:
buttons5 = widgets.Button(description="Avvia task")
buttonl5 = widgets.Button(description="Carica ultima configurazione")
buttonr5 = widgets.Button(description="Reset task")

bllc5 = True
#display(button2, output2)

def run_all5():
    display(Javascript('''
        function sleep2(milliseconds) {
          var start = new Date().getTime();
          for (var i = 0; i < 1e7; i++) {
            if ((new Date().getTime() - start) > milliseconds){
              break;
            }
          }
        }
        var output_area = this;
		// find my cell element
		var cell_element = output_area.element.parents('.cell');
		// which cell is it?
		var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
        try{
            IPython.notebook.execute_cells([cell_idx+1])
			IPython.notebook.execute_cells([cell_idx+2])
			IPython.notebook.execute_cells([cell_idx+3])
        }catch{}
        '''))
    

def on_run_clicked15(b):
    global bllc5
    bllc5=True
    run_all5()
def on_llc5(b):
    global bllc5
    bllc5=False
    run_all5()

def on_reset_clicked5(b):
    global bllc5
    bllc5=True
    run_all5()    
    
buttons5.on_click(on_run_clicked15)
buttonl5.on_click(on_llc5)
buttonr5.on_click(on_reset_clicked5)
widgets.HBox([buttons5, buttonl5, buttonr5])

In [ ]:

    graph_data5 = {'nodes': [{'id': 's', 'label': 's', 'x': 776.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'g', 'label': 'g', 'x': 970.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'e', 'label': 'e', 'x': 826.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'f', 'label': 'f', 'x': 930.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'c', 'label': 'c', 'x': 928.75, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'd', 'label': 'd', 'x': 824.25, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'a', 'label': 'a', 'x': 774.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'b', 'label': 'b', 'x': 968.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'o', 'label': 'o', 'x': 1087.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'n', 'label': 'n', 'x': 1294.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'm', 'label': 'm', 'x': 1142.625, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'l', 'label': 'l', 'x': 1240.125, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'i', 'label': 'i', 'x': 1140.625, 'y': -85.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'h', 'label': 'h', 'x': 1238.125, 'y': -87.25, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'q', 'label': 'q', 'x': 1085.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'p', 'label': 'p', 'x': 1292.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'w', 'label': 'w', 'x': 1461.75, 'y': -250.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 't', 'label': 't', 'x': 1655.75, 'y': -252.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'v', 'label': 'v', 'x': 1511.25, 'y': -185.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'z', 'label': 'z', 'x': 1615.75, 'y': -187.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'r', 'label': 'r', 'x': 1509.25, 'y': -66.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'u', 'label': 'u', 'x': 1613.75, 'y': -68.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'x', 'label': 'x', 'x': 1459.75, 'y': -18.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'y', 'label': 'y', 'x': 1653.75, 'y': -20.5, 'size': 3, 'hidden': False, 'color': '#000000', 'piece': False}, {'id': 'hidden0', 'label': '', 'x': 1245.25, 'y': 90.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden1', 'label': '', 'x': 1421.5, 'y': 18.25, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}, {'id': 'hidden2', 'label': '', 'x': 1047.25, 'y': 105.75, 'size': 3, 'hidden': False, 'color': '#828282', 'enableHovering': False, 'piece': True}], 'edges': [{'id': 'e0', 'source': 's', 'target': 'g', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e1', 'source': 'e', 'target': 'f', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e2', 'source': 'f', 'target': 'd', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e3', 'source': 'd', 'target': 'c', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e4', 'source': 'c', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e5', 'source': 's', 'target': 'e', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e6', 'source': 'g', 'target': 'f', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e7', 'source': 'g', 'target': 'b', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e8', 'source': 's', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e9', 'source': 'a', 'target': 'b', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e10', 'source': 'd', 'target': 'b', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e11', 'source': 'c', 'target': 'a', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e12', 'source': 'o', 'target': 'm', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e13', 'source': 'i', 'target': 'l', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e14', 'source': 'q', 'target': 'i', 'type': 'line', 'label': '5', 'weight': '5', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e15', 'source': 'n', 'target': 'l', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e16', 'source': 'm', 'target': 'h', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e17', 'source': 'p', 'target': 'h', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e18', 'source': 'n', 'target': 'p', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e19', 'source': 'o', 'target': 'q', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e20', 'source': 'b', 'target': 'q', 'type': 'line', 'label': '12', 'weight': '12', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e21', 'source': 'g', 'target': 'o', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e22', 'source': 'p', 'target': 'x', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e23', 'source': 'n', 'target': 'w', 'type': 'line', 'label': '14', 'weight': '14', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e24', 'source': 'w', 'target': 'x', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e25', 'source': 'v', 'target': 'r', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e26', 'source': 'r', 'target': 'u', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e27', 'source': 'u', 'target': 'z', 'type': 'line', 'label': '8', 'weight': '8', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e28', 'source': 'z', 'target': 'v', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e29', 'source': 'w', 'target': 'v', 'type': 'line', 'label': '3', 'weight': '3', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e30', 'source': 'u', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e31', 'source': 'x', 'target': 'y', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e32', 'source': 'y', 'target': 't', 'type': 'line', 'label': '7', 'weight': '7', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e33', 'source': 'z', 'target': 't', 'type': 'line', 'label': '9', 'weight': '9', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e34', 'source': 'x', 'target': 'r', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e35', 'source': 'w', 'target': 't', 'type': 'line', 'label': '4', 'weight': '4', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e36', 'source': 'm', 'target': 'i', 'type': 'arrow', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'e37', 'source': 'l', 'target': 'h', 'type': None, 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': False}, {'id': 'h0', 'source': 'q', 'target': 'hidden0', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h1', 'source': 'hidden0', 'target': 'hidden1', 'type': 'line', 'label': '6', 'weight': '6', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h2', 'source': 'hidden1', 'target': 'n', 'type': 'arrow', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h3', 'source': 'o', 'target': 'hidden2', 'type': 'line', 'label': None, 'weight': None, 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}, {'id': 'h4', 'source': 'hidden2', 'target': 'p', 'type': 'line', 'label': '10', 'weight': '10', 'color': '#828282', 'size': 3, 'hover_color': '#1e6add', 'piece': True}]}
    friends_e5 = {'h0': ['h0', 'h1', 'h2'], 'h1': ['h0', 'h1', 'h2'], 'h2': ['h0', 'h1', 'h2'], 'h3': ['h3', 'h4'], 'h4': ['h3', 'h4']}
    friends_n5 = {'q': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden0': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'hidden1': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n', 'q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'n': ['q', 'hidden0', 'hidden0', 'hidden1', 'hidden1', 'n'], 'o': ['o', 'hidden2', 'hidden2', 'p'], 'hidden2': ['o', 'hidden2', 'hidden2', 'p', 'o', 'hidden2', 'hidden2', 'p'], 'p': ['o', 'hidden2', 'hidden2', 'p']}
    center5 = (388.375, -125.25)
    html_text5 = '''<div>
    <button class="button" id="btn_set_one5">Colora</button>
</div>
<div>
    <button class="button" id="btn_set_two5">Colora</button>
</div>
<div>
    <button class="button" id="btn_unselect_set5">Scolora</button>
</div>
<div>
    <button class="button" id="btn_multiple_sel5">Selezione multipla</button>
</div>
<div>
    <button class="button" id="download_png5">Download Png</button>
</div>
<div>
    <button class="button" id="download_svg5">Download Svg</button>
</div>
<div>
    <button class="button" id="download_adj5">Download Matrice di Adiacenza</button>
</div>
<div>
    <button class="button" id="save5">Salva</button>
</div>'''
    

In [ ]:
#parsing del grafo xml o graphml, viene creato graph_data, original_nodes e original_edges
if bllc5:
    pass
else:
    try:
        list_of_files5 = glob.glob('allegati/ck_points/graph_data_'+str(5)+'_*')
        latest_file5 = max(list_of_files5, key=os.path.getctime)
    except:
        pass
    try:
        with open(latest_file5) as json_file:     
            graph_data5 = json.load(json_file)
    except:
        print("Nessun file di configurazione trovato!")

In [ ]:
js_text_template5 = Template(open('../../graph_utils/js/sigma-graph.js','r').read())
#faccio il dump della struttura che ho creato dentro il js
js_text5 = js_text_template5.substitute({'graph_data': json.dumps(graph_data5),
                                       'exercise' : 5,
                                      'container':'sigma-container5',
                                      'data':0,
                                       'x':center5[0]+center5[0]*0.6,
                                       'y':center5[1],
                                      's':'s5',
                                     'n':'n5',
                                      'ef':'`'+str(friends_e5)+'`',
                                      'nf':'`'+str(friends_n5)+'`',
                                       's_cell': 'cell5'})

html_template5 = Template('''
<style>
    .container { width:100% !important; }
</style>
<link rel="stylesheet" href="../../graph_utils/css/style.css">

    <div class="row row-no-gutters" style="border-color: #1e6add; border-width: 2px;">
        <div class="col-sm-1" style="border-color: #1e6add; border-width: 2px;">
            ''' + html_text5 + '''
        </div>
        <div class="col-sm-11" style="position:border-color: #1e6add; border-width: 2px;">
            <div id="sigma-container5" style="height:750px; width:100%"></div>
        </div>
    </div>

<script> $js_text5 </script>
''')
#render HTML con sostituzione variabili
HTML(html_template5.substitute({'js_text5': js_text5}))

In [ ]:
loader_main()


In [ ]:
import os
import time
from urllib.request import urlopen
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert import RSTExporter, NotebookExporter
from IPython.display import Image, HTML, Javascript
from datetime import datetime

import shutil
last ='preview_last/' #path of last submission
old = 'preview_old/' #path of prevous submissions

#hide alert buttons
def hide_w(w):
    for e in w:
        e.layout.visibility = 'hidden'

#show alert buttons
def show_w(w):
    for e in w:
        e.layout.visibility = 'visible'

def are_you_sure():
    show_w(alert_buttons)

label = widgets.Label(value="Un tuo elaborato era già stato precedentemente prodotto. Vuoi rimpiazzarlo con il presente?")
yes = widgets.Button(description="Si", tooltip="Produce il nuovo elaborato. Esso diviene la versione attuale prendendo il posto del precedente")
no = widgets.Button(description="No")

alert_buttons = [label,yes,no]
hide_w(alert_buttons)

#move all files from last to old and save the new preview
def yes_clicked(b):
    files = os.listdir(last)
    for f in files:
        if f == '.ipynb_checkpoints':
            continue
        if not os.path.isdir('./preview_old'):
            os.mkdir(old)
        shutil.move(last+f, old)
    display(Javascript('IPython.notebook.save_checkpoint();'))
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
    hname = 'Esercizio_04.ipynb'[:-6] + '_' + date_time
    ! jupyter nbconvert Esercizio_04.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.remove_cell_tags='["noexport"]'
    display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    hide_w(alert_buttons)
def no_clicked(b):
    hide_w(alert_buttons)


def generate_preview_HTML(_):
    if not os.path.isdir('./preview_last'):
        os.mkdir(last)
    directory= os.listdir(last)
    if len(directory) <1:
        display(Javascript('IPython.notebook.save_checkpoint();'))
        now = datetime.now() # current date and time
        date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
        hname = 'Esercizio_04.ipynb'[:-6] + '_' + date_time
        ! jupyter nbconvert Esercizio_04.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.remove_cell_tags='["noexport"]'
        display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    else:
        are_you_sure()


button = widgets.Button(description="Salva & Esporta", tooltip="Esporta il foglio Jupyter in HTML nella cartella preview")
output = widgets.Output()
button.on_click(generate_preview_HTML)
yes.on_click(yes_clicked)
no.on_click(no_clicked)
h_box1 = widgets.HBox([button])
h_box2 = widgets.HBox([label,yes,no])
display(widgets.VBox([h_box1,h_box2]))
